In [15]:
import nidaqmx 
import atiiaftt as ati
import time
import numpy as np
from nidaqmx.stream_readers import AnalogMultiChannelReader

In [3]:
CALFILEPATH = "..\FT44764\FT44764.cal"
INDEX = 1

with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai5")
    
    readings = task.read()

sampleTT = [0,0,0,0,0,0]

sensor = ati.FTSensor()
cal = sensor.createCalibration(CALFILEPATH, INDEX)
sensor.setForceUnits("N".encode("utf-8"))
sensor.setTorqueUnits("N-m".encode("utf-8"))
sensor.setToolTransform(sampleTT, "mm".encode("utf-8"), "deg".encode("utf-8"))

# bias = readings[:6]
SG_output = readings[:6]
SG_ref = readings[6:]

# sensor.bias(bias)
output = sensor.convertToFt(SG_output)

print("SG output: ", SG_output)
print("SG red: ", SG_ref)
print("Force: ", output)

DaqError: Device cannot be accessed.  Possible causes:
Device is no longer present in the system.
Device is not powered.
Device is powered, but was temporarily without power.
Device and/or chassis driver support may have been removed.
Device is damaged.
Ensure the device is properly connected and turned on. Ensure the device and/or chassis is supported in the current version of the driver. Check the device's status in NI MAX. Turn the computer off and on again. If you suspect that the device is damaged, contact National Instruments at ni.com/support.
Device Specified: Dev1

Task Name: _unnamedTask<0>

Status Code: -201003

In [ ]:
current_time = time.time()
DURATION = 5
readings_outside_loop = []
with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai5")
    while time.time() < current_time+DURATION:
        readings_outside_loop.append(task.read())

output_outside_loop = sensor.convertToFt(readings_outside_loop[0])
print(len(readings_outside_loop))

In [ ]:
current_time = time.time()
readings_inside_loop = []
while time.time() < current_time+DURATION:
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai5")

        readings_inside_loop.append(task.read())

output_inside_loop = sensor.convertToFt(readings_inside_loop[0])
print(len(readings_inside_loop))

In [14]:
read_task = nidaqmx.Task()
read_task.ai_channels.add_ai_voltage_chan(["Dev1/ai0", ])
reader = AnalogMultiChannelReader(read_task.in_stream,)
values_read = np.zeros(6, dtype=np.float64)

current_time = time.time()
reader.read_one_sample(values_read)
# while time.time() < current_time+DURATION:
    # reader.read_one_sample(values_read)

c:\users\andyb\appdata\local\programs\python\python310\lib\site-packages\nidaqmx\task.py:97: ResourceWarning: Task of name "_unnamedTask<3>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(


DaqError: Device cannot be accessed.  Possible causes:
Device is no longer present in the system.
Device is not powered.
Device is powered, but was temporarily without power.
Device and/or chassis driver support may have been removed.
Device is damaged.
Ensure the device is properly connected and turned on. Ensure the device and/or chassis is supported in the current version of the driver. Check the device's status in NI MAX. Turn the computer off and on again. If you suspect that the device is damaged, contact National Instruments at ni.com/support.
Device Specified: Dev1

Task Name: _unnamedTask<4>

Status Code: -201003